In [53]:
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statistics import mean
from dateutil.relativedelta import relativedelta
import pickle
import os.path
import time

In [58]:
class Stocks:
    """
        Class defining operations for visulazing and valuating specified stocks
    """    
    def __init__(self, ticker: str, time_period: str):
        self.__fid = ticker.upper() + '.stock'
        self._period = time_period
        
        if not os.path.isfile(self.__fid):
            self._ticker = yf.Ticker(ticker)
            with open(self.__fid, 'wb') as f:
                pickle.dump(self._ticker, f)
        else:
            with open(self.__fid, 'rb') as f:
                self._ticker = pickle.load(f)                    
    
    def _get_hist(self):
        """
            Gets Historical Data for Specified Period
        """
        return self._ticker.history(self._period)
        
    def _close_price_analysis(self):
        """
            Extract and Analyze Close Price Data
            @return:
                close_price: Historical Close Price [$]
                close_price_vel: Change in Close Price (Close Price Velocity) [$]
                avg_close_vel: Average Change in Close Price [$]
                num_data: Number of Data Points
        """
        close_price = self._get_hist()['Close']  # Close Prices [$]
        
        num_data = len(close_price)  # Number of Data Points
        
        # Change in Close Price (Velocity of Close Price)
        close_price_vel = np.diff(close_price, n=1, axis=0)  # [$]
        avg_close_vel = mean(close_price_vel)  # Average Close Price Growth [$]
        
        return close_price, close_price_vel, avg_close_vel, num_data
    
    def _close_price_best_fit(self, fitting_data, fit_degree: int):
        """
            Calculates Best-Fit Line for Close Price
            @return: Polynomial describing best-fit curve
        """
        close_price, close_price_vel,\
        avg_close_vel, num_data = self._close_price_analysis()
        
        time_vector = np.linspace(0, num_data, num_data)
        
        # Calculate Polynomial and Coefficients
        poly_coeff = np.polyfit(time_vector, fitting_data, fit_degree)
        poly = np.poly1d(poly_coeff)
        
        return poly
    
    def _eps_predict(self):
        """
            Predict Price using EPS Growth
        """
        trail_eps = self._ticker.info['trailingEps']
        
        peg = self._ticker.info['pegRatio']  # (Price/EPS)/EPS Growth
        
        p_e = self._ticker.info['regularMarketPrice'] / trail_eps  # Price-to-Earnings
        
        eps_growth = p_e / peg  # EPS Growth Rate [percentange]
        
        print(eps_growth)
        
        
        
        

### Testing Functionality:

In [59]:
symbol = 'AAPL'

time_period = 'max'

st = Stocks(symbol, time_period)

In [60]:
close_price, close_prc_vel, avg_close_vel, num_data = st._close_price_analysis()

st._close_price_best_fit(close_price,2);

In [61]:
st._eps_predict()

11.068213664248812
